In [ ]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import glob
import os
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:

 
df = pd.read_csv('dataset.csv', engine='python', encoding='Utf-8')

X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['depressed'], test_size=0.2)
 

In [ ]:
df['tweets'] = df['tweet'].apply(str)


In [ ]:
stopwords = '''
de 
a 
o 
que 
e 
do 
da 
em 
um 
para 
é 
com 
não 
uma 
os 
no 
se 
na 
por 
mais 
as 
dos 
como 
mas 
foi 
ao 
ele 
das 
tem 
à 
seu 
sua 
ou 
ser 
quando 
muito 
há 
nos 
já 
está 
também 
só 
pelo 
pela 
até 
isso 
ela 
entre 
era 
depois 
sem 
mesmo 
aos 
ter 
seus 
quem 
nas 
me 
esse 
eles 
estão 
você 
tinha 
foram 
essa 
num 
nem 
suas 
meu 
às 
minha 
têm 
numa 
pelos 
elas 
havia 
seja 
qual 
será 
nós 
tenho 
lhe 
deles 
essas 
esses 
pelas 
este 
fosse 
dele 
tu 
te 
vocês 
vos 
lhes 
meus 
minhas
teu 
tua
teus
tuas
nosso 
nossa
nossos
nossas
dela 
delas 
esta 
estes 
estas 
aquele 
aquela 
aqueles 
aquelas 
isto 
aquilo 
estou
está
estamos
estão
estive
esteve
estivemos
estiveram
estava
estávamos
estavam
estivera
estivéramos
esteja
estejamos
estejam
estivesse
estivéssemos
estivessem
estiver
estivermos
estiverem
hei
há
havemos
hão
houve
houvemos
houveram
houvera
houvéramos
haja
hajamos
hajam
houvesse
houvéssemos
houvessem
houver
houvermos
houverem
houverei
houverá
houveremos
houverão
houveria
houveríamos
houveriam
sou
somos
são
era
éramos
eram
fui
foi
fomos
foram
fora
fôramos
seja
sejamos
sejam
fosse
fôssemos
fossem
for
formos
forem
serei
será
seremos
serão
seria
seríamos
seriam
tenho
tem
temos
tém
tinha
tínhamos
tinham
tive
teve
tivemos
tiveram
tivera
tivéramos
tenha
tenhamos
tenham
tivesse
tivéssemos
tivessem
tiver
tivermos
tiverem
terei
terá
teremos
terão
teria
teríamos
teriam
pra
O
q
https
http
A
tá
pq
pic
E
'''
stopwords = re.split(r'\W', stopwords)

In [ ]:
import nltk
import string
from nltk.tokenize import word_tokenize

review_lines = list()
lines = df['tweet'].values.tolist()

for line in lines:   
    tokens = word_tokenize(str(line))
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word    
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words    
    words = [w for w in words if not w in stopwords]
    review_lines.append(words)

In [ ]:
len(review_lines)


In [ ]:
import gensim 

EMBEDDING_DIM = 100
# train word2vec model
model = gensim.models.Word2Vec(sentences=review_lines, size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
# vocab size
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

In [ ]:
filename = 'tweets_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [ ]:
model.wv.most_similar('mal')#, topn =1)


In [ ]:

import os

embeddings_index = {}
f = open(os.path.join('', 'tweets_embedding_word2vec.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['tweet'],df['depressed'], test_size=0.2)

In [ ]:
total_reviews = X_train + X_test
max_length = 100 # try other options like mean of sentence lengths

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

VALIDATION_SPLIT = 0.2

# vectorize the text samples into a 2D integer tensor
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(review_lines)
sequences = tokenizer_obj.texts_to_sequences(review_lines)

# pad sequences
word_index = tokenizer_obj.word_index
print('Found %s unique tokens.' % len(word_index))

review_pad = pad_sequences(sequences, maxlen=max_length)
sentiment =  df['depressed'].values
print('Shape of review tensor:', review_pad.shape)
print('Shape of sentiment tensor:', sentiment.shape)

# split the data into a training set and a validation set
indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation_samples = int(VALIDATION_SPLIT * review_pad.shape[0])

X_train_pad = review_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
X_test_pad = review_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

In [ ]:
print('Shape of X_train_pad tensor:', X_train_pad.shape)
print('Shape of y_train tensor:', y_train.shape)

print('Shape of X_test_pad tensor:', X_test_pad.shape)
print('Shape of y_test tensor:', y_test.shape)

In [ ]:
EMBEDDING_DIM =100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
print(num_words)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.initializers import Constant

# define model
model = Sequential()
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_length,
                            trainable=False)

model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

print(model.summary())

# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the model
model.fit(X_train_pad, y_train, batch_size=128, epochs=20, validation_data=(X_test_pad, y_test), verbose=2)